# 유의어 생성

In [ ]:
import transformers
import torch

model_id = "rtzr/ko-gemma-2-9b-it"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)
pipeline.model.eval()

In [2]:
def request_prompt(message, max_new_tokens=100, temperature=None):
    try:
        prompt = pipeline.tokenizer.apply_chat_template(
            message, 
            tokenize=False, 
            add_generation_prompt=True
        )

        terminators = [
            pipeline.tokenizer.eos_token_id,
            pipeline.tokenizer.convert_tokens_to_ids("<end_of_turn>")
        ]

        do_sample = True if temperature else False
        
        outputs = pipeline(
            prompt,
            max_new_tokens=max_new_tokens,
            eos_token_id=terminators,
            do_sample=do_sample,
            temperature=temperature,
        )
        return outputs[0]["generated_text"][len(prompt):]
    except Exception as e:
        print(e)
        return ""

In [7]:
import gc
import json

persona = f"""사용자 입력 문장에서 핵심 단어를 뽑고, 각 단어별 synonym을 5개 뽑아서 array로 반환해주세요.
**출력 형식**:[["이태리 언론", "이탈리아 언론", "이탈리아 미디어", "이탈리아 뉴스", "이탈리아 출판"], ["자유", "권리", "해방", "독립", "자율성"]]
**주의 사항**: 배열만 출력하세요.
"""

def generate_data(input_filename, output_filename):
    print(f">> {input_filename}")

    with open(input_filename) as f, open(output_filename, "w") as of:
        idx = 0

        for line in f:
            # print(f'[{idx}]')
            # if idx > 30:
            #   break
            
            j = json.loads(line)
            title = j["title"]
            synonyms = []

            try:
                prompt = [{"role": "system", "content": persona}] + [{"role": "user", "content": title}]
                # print(f'prompt : {prompt}')

                response = request_prompt(prompt, 500)
                # print(f'plain : {response}')
                
                synonyms = response

            except Exception as e:
                id = j["docid"]
                print(f"++ Error! {id} : {title} : {e}")
                print(f'plain response : {response}')                

            output = {
                "title": j["title"],
                "synonyms": synonyms,
                "docid": j["docid"],
            }
            print(f'{json.dumps(output, ensure_ascii=False)}')

            # 수정된 JSON을 출력 파일에 저장
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')

            idx += 1

            gc.collect()
            torch.cuda.empty_cache()

input = "./data/documents_meta.jsonl"
output = "./data/synonyms.jsonl"

generate_data(input, output)

>> ./data/documents_meta.jsonl
{"title": "트로이군 소행성: 목성의 중력 궤도 안에 위치한 소행성", "synonyms": "[[\"트로이군 소행성\", \"그리스형 소행성\", \"목성 소행성\", \"목성 궤도 소행성\", \"목성 주위 소행성\"], [\"목성\", \"행성\", \"거대 행성\", \"천체\", \"태양계\"]] \n", "docid": "d525c38e-a296-4dec-8a60-d1fb98a355e3"}
{"title": "드미트리 이바노프스키: 최초의 바이러스 발견", "synonyms": "[[\"드미트리 이바노프스키\", \"이바노프스키\", \"드미트리\", \"이바노프\", \"과학자\"], \n[\"바이러스\", \"바이러스 감염\", \"전염병\", \"독감\", \"바이러스 질환\"]] \n", "docid": "38686456-b993-4cbb-af0d-1c53df2f3e12"}
{"title": "이탈리아인들의 자유로운 미디어에 대한 강한 지지", "synonyms": "[[\"이탈리아인\", \"이탈리아 국민\", \"이탈리아인들\", \"이탈리아 거주자\", \"이탈리아 사회\"], [\"미디어\", \"언론\", \"신문\", \"방송\", \"출판\"], [\"자유로운\", \"자유\", \"해방\", \"독립\", \"권리\"], [\"지지\", \"지원\", \"옹호\", \"후원\", \"인식\"]] \n", "docid": "91a2fe6f-a626-499e-8111-919e7620ca90"}
{"title": "채식주의와 체질량 지수(BMI)의 관계", "synonyms": "[[\"채식주의\", \"식물성 식단\", \"채식\", \"비육식\", \"건강 식단\"], [\"체질량 지수\", \"BMI\", \"체중 지수\", \"신체 지수\", \"체지방 비율\"]] \n", "docid": "176c3a3a-f183-401b-811e-5381a0f63656"}
{"

In [14]:
import json
import ast

def modify(line):
    first_list_str = None

    # JSON 데이터 로드
    data = json.loads(line.strip())

    # JSON 문자열을 로드하여 딕셔너리로 변환
    if isinstance(line, str):
        data = json.loads(line.strip())
    else:
        data = line 

    synonyms_str = data["synonyms"]    

    # 문자열에서 첫 번째 배열 추출
    if isinstance(synonyms_str, str):
        # 첫 번째 닫는 대괄호 인덱스 찾기
        end_idx = synonyms_str.find("]")

        if end_idx != -1:
            # 끝 대괄호부터 거꾸로 첫 번째 대괄호 인덱스 찾기
            start_idx = synonyms_str.rfind("[", 0, end_idx)

            if start_idx != -1:
                # 첫 번째 대괄호 내의 내용 추출
                first_list_str = synonyms_str[start_idx:end_idx + 1]
                
                # 이스케이프 제거
                first_list_str = first_list_str.replace('\\"', '').replace('"', '')
            else:
                print("첫 번째 대괄호를 찾을 수 없습니다.")
        else:
            print("마지막 대괄호를 찾을 수 없습니다.")
    else:
        print("synonyms_str는 문자열이 아닙니다:", type(synonyms_str))

    return first_list_str
    
def transform_jsonl(input_jsonl_path, output_jsonl_path):
    success = 0
    fail = 0

    try:
        with open(input_jsonl_path, 'r', encoding='utf-8') as infile, open(output_jsonl_path, 'w', encoding='utf-8') as outfile:
            for line in infile:
                try:
                    data = json.loads(line.strip())
                    synonyms_str = data["synonyms"]

                    # synonyms 필드가 존재하는지 확인
                    if "synonyms" in data:
                        synonyms_str = data["synonyms"]
                        
                        # synonyms가 문자열인 경우 파이썬 리스트로 변환
                        synonyms_list = ast.literal_eval(synonyms_str)
                        
                        # 리스트의 각 항목을 콤마로 연결하여 한 줄로 변환
                        result_synonyms = [', '.join(map(str, row)) for row in synonyms_list]
                        
                        # 변환된 synonyms를 다시 딕셔너리에 저장
                        data["synonyms"] = result_synonyms
                    
                    # 변환된 데이터를 출력 파일에 기록
                    outfile.write(json.dumps(data, ensure_ascii=False) + '\n')
                    success = success+1
                
                except Exception as e:
                    try:
                        data["synonyms"] = modify(line)
                        print(data["synonyms"])

                        outfile.write(json.dumps(data, ensure_ascii=False) + '\n')
                        success = success+1
                    except Exception as e:
                        fail = fail+1
                        print(f"Unknown error at line {e}: {str(e)}")
                        print(f"Failed line content: {line.strip()}")

                    continue
                
        print(f"변환이 완료되었습니다. 결과가 {output_jsonl_path}에 저장되었습니다.")
    
    except Exception as e:
        fail = fail+1
        print(f"Unknown error at line {e}: {str(e)}")
        print(f"Failed line content: {line.strip()}")                    

    print(f"성공 : {success}, 실패 : {fail}")

# 입력 및 출력 파일 경로
output_jsonl_path = './data/synonyms_cleaned.jsonl'

# 변환 실행
transform_jsonl(output, output_jsonl_path)

[철 상태 평가, 빈혈 진단, 철분 결핍 검사, 헤모글로빈 수치 평가, 혈액 검사]
[대서양, 아틀란티스, 대서양 해역, 아틀란틱, 해양]
[ALU, Arithmetic Logic Unit, Arithmetic Processing Unit, Arithmetic and Logic Unit, Processing Unit]
[컴파일러 생성 객체 모듈, 컴파일러 생성 도구, 컴파일러 구조체, 컴파일러 인터페이스, 컴파일러 엔진]
[풍력 발전소, 풍력 터빈, 풍력 에너지, 풍력 발전, 풍력 시설]
[기후변화, 기후 변화, 기온 상승, 온난화, 지구 온난화]
[암모니아, 질소화합물, NH3, 암모니아 가스, 암모니아 수용액]
[]
[물체, 대상, 구체, 체, 사물]
[]
[설탕, 당분, 설탕분말, 사카로스, 글루코스]
[이태리 언론, 이탈리아 언론, 이탈리아 미디어, 이탈리아 뉴스, 이탈리아 출판]
[apple, banana, cherry, apple]
[최대 동적 운동 시간 주기, 최대 운동 시간 주기, 동적 운동 시간 주기, 운동 시간 주기, 동적 활동 시간 주기]
[이태리 언론, 자유]
[컴파일러, 컴파일러 프로그램, 번역기, 코드 변환기, 프로그램 생성 도구]
[치매, 치매증, 인지기능장애, 퇴행성 뇌 질환, 기억력 저하]
[상자, 박스, 장갑, 컨테이너, 화물]
[이상 용액, 이상 혼합물, 완벽 혼합물, 용매 간섭 없는 혼합물,  आदर्श 용액]
[무중력, 중력이 없는, 무중력 환경, 저중력, 중력 없는 공간]
마지막 대괄호를 찾을 수 없습니다.
None
[::-1]
[프로그래밍, 할당문, 실행, 시간, 계산]
[석유, 원유, 석탄, 에너지, 화석 연료]
[경석, 돌, 암석, 바위, 광물]
[발진기, 발진기, 발진회로, 발진기, 오실레이터]
[지하 서식지, 지하 굴, 터널, 동굴, 보금자리]
[니켈(II)니트레이트, 니켈(II)염화물, 니켈니트레이트, 니켈염, 니켈 화합물]
[뉴클레오티드 변이, DNA 변이, 염기 변이, 유전자 변이, 유전자 돌연변이]
[유닉

In [15]:
import json

def extract_and_save_synonyms(jsonl_file_path, output_file_path, max_words, max_synonyms):
    # JSONL 파일을 읽고 유의어를 추출하여 텍스트 파일에 저장
    with open(jsonl_file_path, 'r', encoding='utf-8') as jsonl_file, open(output_file_path, 'w', encoding='utf-8') as txt_file:
        for line in jsonl_file:
            try :
                data = json.loads(line)
                synonyms = data.get("synonyms", [])[:max_words]

                for synonym_list in synonyms:
                    # 유의어 목록에서 지정한 수의 유의어를 가져오기
                    selected_synonyms = synonym_list.split(", ")[:max_synonyms]
                    txt_file.write(", ".join(selected_synonyms) + "\n")  # 선택한 유의어를 쉼표로 구분하여 한 줄에 저장
            
            except Exception as e:
                print(f"{data}, {e}")
                continue

# 예시 사용
jsonl_file_path = './data/synonyms_cleaned.jsonl' # 입력 JSONL 파일 경로
output_file_path = './data/synonyms.txt'    # 출력 TXT 파일 경로
max_words = 1  # 가져올 단어의 최대 개수
max_synonyms = 10  # 가져올 유의어의 최대 개수
extract_and_save_synonyms(jsonl_file_path, output_file_path, max_words, max_synonyms)

{'title': '야구공의 운동량 계산', 'synonyms': None, 'docid': '580ab9c3-27e8-49cf-b005-af2f97a47279'}, 'NoneType' object is not subscriptable
{'title': '시간에 따른 가속도를 가진 물체의 이동 거리 계산', 'synonyms': None, 'docid': '7dfb303e-e0d0-49ef-afd4-29c2ade7d1e7'}, 'NoneType' object is not subscriptable
{'title': '볼링공의 평균 가속도 계산', 'synonyms': None, 'docid': '33625035-8fd5-49f2-9c7c-b0dde8b1c746'}, 'NoneType' object is not subscriptable
{'title': 'N이 0인 경우 평균 계산 오류 처리', 'synonyms': None, 'docid': 'ec87a926-171d-4f62-9acc-1b870c010a16'}, 'NoneType' object is not subscriptable
{'title': '숫자 맞추기 게임', 'synonyms': None, 'docid': '56658a7c-2f09-4754-8515-fe7c1f569d7c'}, 'NoneType' object is not subscriptable
{'title': '16진수를 이진수로 변환하기', 'synonyms': None, 'docid': 'ed600c5d-9489-437b-bec2-f22a14347b07'}, 'NoneType' object is not subscriptable
{'title': '해시 테이블 버킷 번호 2의 내용', 'synonyms': None, 'docid': 'f83fbad4-0da0-42a3-bd0c-8d7d7afc1f57'}, 'NoneType' object is not subscriptable
{'title': '멕시코계 미국인 및 아프리카계 미국인 남성의 성관계

### 클러스터링